In [10]:
from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    FileType as ft,
    AnnDataWrapper,
    OmeZarrWrapper,
)
from os.path import join

In [11]:
vc = VitessceConfig(schema_version="1.0.15", name='10x genomics', description='Visium')

In [12]:
dataset = vc.add_dataset(name='Human lymph node').add_object(AnnDataWrapper(
    adata_path=join("processed_data", "human_lymph_node.adata.zarr"),
    obs_embedding_paths=["obsm/X_umap", "obsm/X_pca"],
    obs_embedding_names=["UMAP", "PCA"],
    obs_locations_path="obsm/spatial",
    obs_segmentations_path="obsm/segmentations",
    obs_set_paths=["obs/clusters"],
    obs_set_names=["Leiden Cluster"],
    obs_feature_matrix_path="obsm/X_hvg",
    feature_filter_path="var/highly_variable",
    coordination_values={
        "obsType": "spot",
        "featureType": "gene",
        "featureValueType": "expression"
    }
)).add_object(OmeZarrWrapper(
    img_path=join("processed_data", "human_lymph_node.ome.zarr")
))

In [13]:
spatial_colored_by_cluster = vc.add_view(cm.SPATIAL, dataset=dataset)
spatial_colored_by_expression = vc.add_view(cm.SPATIAL, dataset=dataset)

layer_controller = vc.add_view(cm.LAYER_CONTROLLER, dataset=dataset).set_props(disableChannelsIfRgbDetected=True)
spot_set_manager = vc.add_view(cm.OBS_SETS, dataset=dataset)
gene_list = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
heatmap = vc.add_view(cm.HEATMAP, dataset=dataset).set_props(transpose=True)

In [14]:
vc.layout(
    (spatial_colored_by_cluster | spatial_colored_by_expression)
    / ((layer_controller | spot_set_manager) | (gene_list | heatmap))
);

In [15]:
spatial_views = [
    spatial_colored_by_cluster,
    spatial_colored_by_expression,
    layer_controller,
]
all_views = [
    *spatial_views,
    spot_set_manager,
    gene_list,
    heatmap
]
vc.link_views(
    all_views,
    [ct.OBS_TYPE],
    ["spot"]
)

spatial_segmentation_layer_value = {
    "radius": 65,
    "stroked": True,
    "visible": True,
    "opacity": 1
}
spatial_image_layer_value = [{
    "type": "raster",
    "index": 0,
    "colormap": None,
    "transparentColor": None,
    "opacity": 1,
    "domainType": "Min/Max",
    "channels": [
        {
          "selection": { "c": 0 },
          "color": [255, 0, 0],
          "visible": True,
          "slider": [0, 255]
        },
        {
          "selection": { "c": 1 },
          "color": [0, 255, 0],
          "visible": True,
          "slider": [0, 255]
        },
        {
          "selection": { "c": 2 },
          "color": [0, 0, 255],
          "visible": True,
          "slider": [0, 255]
        }
    ]
}]
vc.link_views(
    spatial_views,
    [ct.SPATIAL_IMAGE_LAYER, ct.SPATIAL_SEGMENTATION_LAYER],
    [spatial_image_layer_value, spatial_segmentation_layer_value]
)
vc.link_views(
    spatial_views,
    [ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y],
    [-2.6, 1000, 1000]
)
vc.link_views(
    [spatial_colored_by_cluster, heatmap, spot_set_manager],
    [ct.OBS_COLOR_ENCODING],
    ["cellSetSelection"]
)
vc.link_views(
    [spatial_colored_by_expression, gene_list],
    [ct.OBS_COLOR_ENCODING, ct.FEATURE_SELECTION],
    ["cellSetSelection", ["CR2"]]
)

In [16]:
vw = vc.widget()
vw

VitessceWidget(config={'version': '1.0.15', 'name': '10x genomics', 'description': 'Visium', 'datasets': [{'ui…